In [1]:
%autosave 0

Autosave disabled


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from adam_acquire import get_titanic

Let's summarize our dataframe with some familiar methods and attributes including:
- .head()
- .shape
- .describe()

In [10]:
df = get_titanic()

Let's take a look at each column in isolation. We are allowed to do this before splitting our data!

In [12]:
for col in df.columns[num_mask]:
    # plt figure will plot each loop on a differnt figure
    plt.figure()
    plt.hist(df[col])
    # This will lable each progromatically
    plt.title(f'Distribution of {col})

SyntaxError: unterminated string literal (detected at line 5) (2690598573.py, line 5)

Let's count how many null values are present in each column.

In [13]:
df.isna()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,False,False,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,False,False,False,False,False,False,False,False,False,True,False,False
887,False,False,False,False,False,False,False,False,False,False,False,False,False
888,False,False,False,False,True,False,False,False,False,False,True,False,False
889,False,False,False,False,False,False,False,False,False,False,False,False,False


In [14]:
# This will tell you which collumns have nulls and how many
df.isna().sum()

passenger_id      0
survived          0
pclass            0
sex               0
age             177
sibsp             0
parch             0
fare              0
embarked          2
class             0
deck            688
embark_town       2
alone             0
dtype: int64

What to keep and what to drop after initial summarization.

- Dropping passenger ID because it doesn't hold value. More serves as an index and not predictive of anything
- Dropping embraked beacsue it contains redudntant information. 
- - Drop deck becasue it contains many null values
- Droping pclass because it contains redundant inforamtion to class.
- Will need to clean up embark town and age becasue of null values

In [17]:
df = df.drop(columns = ['passenger_id' , 'pclass', 'embarked'])
df.head()

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,NaN,Southampton,0
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,0
2,1,female,26.0,0,0,7.9250,Third,NaN,Southampton,1
3,1,female,35.0,1,0,53.1000,First,C,Southampton,0
4,0,male,35.0,0,0,8.0500,Third,NaN,Southampton,1


Let's split our data into train, validate, and test subsets before determining how to impute for null values. Otherwise this would be considered data leakage!

In [18]:
seed = 42
train, val_test = train_test_split(df,train_size = 0.7, 
                                   random_state = seed,
                                  stratify = df.survived)

val, test = train_test_split(val_test, train_size = 0.5 ,
                            random_state = seed,
                            stratify = val_test.survived)

In [19]:
train.survived.value_counts(normalize = True)

survived
0    0.616372
1    0.383628
Name: proportion, dtype: float64

In [21]:
val.survived.value_counts(normalize = True)

survived
0    0.61194
1    0.38806
Name: proportion, dtype: float64

Now that we have split our data, let's figure out how to deal with those pesky null values.

In [22]:
train.isna().sum()

survived         0
sex              0
age            119
sibsp            0
parch            0
fare             0
class            0
deck           488
embark_town      2
alone            0
dtype: int64

In [34]:
train.embark_town.value_counts()
# use .replace to 

embark_town
Southampton     444
Cherbourg       124
Queenstown       53
Southhampton      2
Name: count, dtype: int64

In [33]:
train.embark_town = train.embark_town.fillna('Southampton')

We can also use the SimpleImputer object instead of manual imputation. Both are valid!

In [36]:
train[train.age.isna()].head(10)

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,alone
45,0,male,NaN,0,0,8.0500,Third,NaN,Southampton,1
28,1,female,NaN,0,0,7.8792,Third,NaN,Queenstown,1
633,0,male,NaN,0,0,0.0000,First,NaN,Southampton,1
727,1,female,NaN,0,0,7.7375,Third,NaN,Queenstown,1
740,1,male,NaN,0,0,30.0000,First,D,Southampton,1
64,0,male,NaN,0,0,27.7208,First,NaN,Cherbourg,1
656,0,male,NaN,0,0,7.8958,Third,NaN,Southampton,1
303,1,female,NaN,0,0,12.3500,Second,E,Queenstown,1
527,0,male,NaN,0,0,221.7792,First,C,Southampton,1
359,1,female,NaN,0,0,7.8792,Third,NaN,Queenstown,1


In [37]:
train.age.describe()

count    504.000000
mean      30.045476
std       14.502377
min        0.420000
25%       21.000000
50%       29.000000
75%       39.000000
max       80.000000
Name: age, dtype: float64

In [ ]:
# going ot impute the median age

In [38]:
imputer = SimpleImputer(strategy = 'median')
imputer.fit(train[['age']])

SimpleImputer(strategy='median')

In [41]:
train.age = imputer.transform(train[['age']])
train.age.isna().sum()

0

In [46]:
val.age.isna().sum()

29

In [45]:
imputer.transform(train[['age']])
train.age.isna().sum()

0

Let's turn our work into functions! We can use a series of functions to quickly acquire and prepare our data!

In [47]:
# appropriate for classification becasue of stratify?
def train_val_test(df, strat, seed = 55):
    train, val_test = train_test_split(df, train_size = 0.7,
                                      random_state = seed,
                                      stratify = df[strat])
    val, test = train_test_split(val_test, train_size=0.5,
                                random_state = seed,
                                stratify = val_test[strat])
    return train, val, test

In [48]:
ftrain, fval, ftest = train_val_test(df,'survived')

In [49]:
ftrain

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,alone
519,0,male,32.0,0,0,7.8958,Third,NaN,Southampton,1
441,0,male,20.0,0,0,9.5000,Third,NaN,Southampton,1
43,1,female,3.0,1,2,41.5792,Second,NaN,Cherbourg,0
341,1,female,24.0,3,2,263.0000,First,C,Southampton,0
664,1,male,20.0,1,0,7.9250,Third,NaN,Southampton,0
...,...,...,...,...,...,...,...,...,...,...
314,0,male,43.0,1,1,26.2500,Second,NaN,Southampton,0
639,0,male,NaN,1,0,16.1000,Third,NaN,Southampton,0
92,0,male,46.0,1,0,61.1750,First,E,Southampton,0
355,0,male,28.0,0,0,9.5000,Third,NaN,Southampton,1


In [50]:
train.head()

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,alone
748,0,male,19.0,1,0,53.1000,First,D,Southampton,0
45,0,male,29.0,0,0,8.0500,Third,NaN,Southampton,1
28,1,female,29.0,0,0,7.8792,Third,NaN,Queenstown,1
633,0,male,29.0,0,0,0.0000,First,NaN,Southampton,1
403,0,male,28.0,1,0,15.8500,Third,NaN,Southampton,0
